In [ ]:
import numpy as np
import os
import h5py, pickle
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cB211.072.64'

ens2paths={
    'cB211.072.64':['/onyx/qdata/ikoutsou/gffs-analysis/cB211.072.64/h5data/','/onyx/qdata/ckummer/juwels/run2/','/onyx/qdata/ckummer/juwels/run3/'],
    'cC211.060.80':['/onyx/qdata/ckummer/juwels/Censemble/oneD/'],
    'cD211.054.96':['/onyx/qdata/ckummer/juwels/Densemble/oneD/'],
}

savepath=f'/nvme/h/cy22yl1/projectData/05_moments/{ens}/data_merge/'
os.makedirs(savepath,exist_ok=True)

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]
projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
inds_trace=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]

xyzt2xyz0=lambda x: x if x!='t' else '0'
t=[insert for insert in inserts]; inserts_key=[f'=der:g{xyzt2xyz0(insert[1])}D{xyzt2xyz0(insert[0])}:sym=' for insert in t]

elements=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in permutations([0, 1, 2], 3)] # Permute first Flip next
def rotate_mom(e,mom):
    sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
    return [sx*mom[iix],sy*mom[iiy],sz*mom[iiz],sx*mom[iix+3],sy*mom[iiy+3],sz*mom[iiz+3]]
def rotate_vec3(e,vec3): #xyzt=0123
    if vec3 in ['t']:
        return (1,vec3)
    sx,sy,sz,xyz=e; ivec3={'x':0,'y':1,'z':2}[vec3]; ivec3_new=xyz[ivec3]; vec3_new=['x','y','z'][ivec3_new]
    sign=[sx,sy,sz][ivec3_new]
    return (sign,vec3_new)
def rotate_proj(e,proj):
    if proj=='P0':
        return (1,proj)
    sx,sy,sz,xyz=e; det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
    (sign,proj_new)=rotate_vec3(e,proj[-1])    
    return (sign*det,proj_new)
def rotate_insert(e,insert):
    s1,i1=rotate_vec3(e,insert[0]); s2,i2=rotate_vec3(e,insert[1])
    return (s1*s2,i1+i2 if i1+i2 in inserts else i2+i1)

def mom2moms(mom):
    moms=list(set([tuple(rotate_mom(e,mom)) for e in elements])) 
    moms.sort()
    moms=np.array(moms)
    return moms
def mom2standard(mom):
    moms=list(set([tuple(rotate_mom(e,mom)) for e in elements])) 
    return list(max(moms, key=lambda x: x[::-1]))

def mom2name(mom):
    assert(np.all(mom==mom2standard(mom)))
    return ','.join([str(ele) for ele in mom])

In [42]:
path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

def extractFile(f,mom):    
    p1=mom[:3]; q=mom[3:]
    assert(np.all(p1==[0,0,0]))
    msq='msq{:04}'.format(sum([x**2 for x in q]))
    moms=mom2moms(mom)
    
    t=list(f.keys()); t=[ele for ele in t if ele not in ['ffs']]; assert(len(t)==1); Nsrc_key=t[0]; Nsrc=int(Nsrc_key[4:])
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}'].keys()); assert(len(t)==1); proj_key=t[0]
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}'].keys()); assert(len(t)==1); tf_key=t[0]
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}/{tf_key}'].keys()); assert(len(t)==1); ud_key=t[0]
    moms_old=f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}/{tf_key}/{ud_key}/mvec'][:]
    dic={}
    for i,m in enumerate(moms_old):
        dic[tuple(m)]=i
    inds=[dic[tuple(m[3:])] for m in moms]
    
    trajs=[traj.decode() for traj in f[Nsrc_key][msq][inserts_key[0]][proj_key][tf_key][ud_key]['trajs'][:]]
    if trajs[0][1]=='-':
        dic={}
        for i,tra in enumerate(trajs):
            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
            dic[cfg]=i
    if trajs[0][-2]=='r':
        dic={}
        for i,tra in enumerate(trajs):
            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
            dic[cfg]=i
    inds_cfg=[dic[cfg] for cfg in cfgs]
    
    dat=np.array([f[f'{Nsrc_key}/{msq}/{insert_key}/{proj_key}/{tf_key}/{ud_key}/arr'][:,:,inds] for insert_key in inserts_key])
    dat=dat[:,inds_cfg]
    
    trace=np.sum(dat[inds_trace],axis=0)
    dat[inds_trace] -= trace[None,...]/4
    return dat

avgmoreQ=True
def convertData(dat,mom):
    dat=np.transpose(dat,[2,3,4,0,1])
    
    if avgmoreQ:
        assert(np.all(mom==mom2standard(mom)))
        moms=mom2moms(mom)
        dic={}
        for i,m in enumerate(moms):
            dic[tuple(m)]=i

        e2ind_mom={}; e2inds_proj={}; e2signs_proj={}; e2inds_insert={}; e2signs_insert={}
        for e in elements:
            e2ind_mom[e]=dic[tuple(rotate_mom(e,mom))]
            
            sx,sy,sz,xyz=e; signs=[sx,sy,sz,1]
            ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
            xyzt=['x','y','z','t']
            xyzt2={'x':xyzt[ix],'y':xyzt[iy],'z':xyzt[iz],'t':'t'}
            det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
            
            e2signs_proj[e]=np.array([1,sx*det,sy*det,sz*det])
            e2inds_proj[e]=[0,ix+1,iy+1,iz+1]
            
            e2signs_insert[e]=np.array([signs[xyzt.index(insert[0])]*signs[xyzt.index(insert[1])] for insert in inserts])
            e2inds_insert[e]=[xyzt2[insert[0]]+xyzt2[insert[1]] for insert in inserts]
            e2inds_insert[e]=[inserts.index(ele) if ele in inserts else inserts.index(ele[1]+ele[0]) for ele in e2inds_insert[e]]
            
        def get(e):
            t=dat[:,:,e2ind_mom[e]]
            
            t=t*e2signs_proj[e][None,None,:,None]
            t=t[:,:,e2inds_proj[e]]
            
            t=t*e2signs_insert[e][None,None,None,:]
            t=t[:,:,:,e2inds_insert[e]]
            
            return t
        dat=np.mean([get(e) for e in elements],axis=0)[:,:,None,:,:]
        
    return dat

In [ ]:
paths=ens2paths[ens]

tf2Nsrcs={}
for ipath,path in enumerate(paths):
    files=os.listdir(path)
    tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
    tfs=list(set(tfs)); tfs.sort()
    
    for tf in tfs:
        tf_str='{:02}'.format(tf)
        for proj,proj_name in zip(projs[:1],['P0']):
            with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu:
                if tf not in tf2Nsrcs:
                    tf2Nsrcs[tf]=np.zeros(len(paths),dtype=int)
                Nsrc=int(list(fu.keys())[0][4:])
                tf2Nsrcs[tf][ipath]=Nsrc  
tfs=list(tf2Nsrcs.keys()); tfs.sort()

def run(mom):
    tf2dat={(tf,j):0 for tf in tf2Nsrcs.keys() for j in ['j+','j-']}
    for ipath,path in enumerate(paths):
        files=os.listdir(path)
        tfs=[int(file.split('-')[2][2:]) for file in files if file.startswith('thrp-oneD')]
        tfs=list(set(tfs)); tfs.sort()
        
        for tf in tfs:
            tf_str='{:02}'.format(tf)
            
            tu=[]; td=[]
            for iproj,proj,proj_name in zip([0,1,2,3],projs,['P0','P4','P5','P6']):
                print(tf,proj,end='                 \r')
                with h5py.File(f'{path}thrp-oneD-dt{tf_str}-up-{proj_name}.h5') as fu, h5py.File(f'{path}thrp-oneD-dt{tf_str}-dn-{proj_name}.h5') as fd:
                    
                    tu.append(extractFile(fu,mom))
                    td.append(extractFile(fd,mom))
            tu=convertData(tu,mom); td=convertData(td,mom)
            tp=tu+td; tm=tu-td
            
            weight=tf2Nsrcs[tf][ipath]/np.sum(tf2Nsrcs[tf])
            tf2dat[(tf,'j+')]+= tp*weight
            tf2dat[(tf,'j-')]+= tm*weight
    
    path=f'{savepath}conn_{mom2name(mom)}.h5'
    with h5py.File(path,'w') as f:
        f.create_dataset('notes',data=['cfg,time,mom,proj,insert','mom=[sink,ins]; sink+ins=src','proj=[P0,Px,Py,Pz]'])
        f.create_dataset('inserts',data=inserts)
        f.create_dataset('cfgs',data=cfgs)
        f.create_dataset('moms',data=[mom] if avgmoreQ else mom2moms(mom))
        
        for key in tf2dat.keys():
            tf,j=key
            f.create_dataset(f'data/{j};conn_{tf}',data=tf2dat[key])

run([0,0,0,0,0,0])

# correlated 2pt

In [ ]:
avgmoreQ_2pt=True
assert(avgmoreQ_2pt) # otherwise the memory is not enough to handle
def run(moms):
    tf2dat={tf:0 for tf in tf2Nsrcs.keys()}
    for ipath,path in enumerate(paths):
        file=f'{path}twop.h5'
        with h5py.File(file) as f:         
            for tf in tf2Nsrcs.keys():
                print(tf,ipath,end='                   \r')
                Nsrc=tf2Nsrcs[tf][ipath]
                
                if Nsrc!=0:
                    Nsrc_key=f'nsrc{Nsrc}'
                    if list(f.keys())[0].startswith('dt'):
                        Nsrc_key=f'dt{tf:02}/'+Nsrc_key
                    
                    trajs=[traj.decode() for traj in f[f'{Nsrc_key}/msq0000/ppm/fwd/trajs'][:]]
                    if trajs[0][1]=='-':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
                            dic[cfg]=i
                    if trajs[0][-2]=='r':
                        dic={}
                        for i,tra in enumerate(trajs):
                            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
                            dic[cfg]=i
                    inds_cfg=[dic[cfg] for cfg in cfgs]
                    
                    def get(mom):
                        msq='msq{:04}'.format(mom[0]**2+mom[1]**2+mom[2]**2)
                        tmoms=f[f'{Nsrc_key}/{msq}/ppm/fwd/mvec'][:]
                        inds=[i for i,m in enumerate(tmoms) if np.all(mom2standard([0,0,0]+list(m))==mom2standard([0,0,0]+list(mom)))]
                        t=f[f'{Nsrc_key}/{msq}/ppm/fwd/arr'][:]
                        if len(t.shape)==3:
                            t=t[:,:,inds]
                        else:
                            t=t[:,None,:]
                            t=t[:,[0]*(tf+1),:]
                            t=t[:,:,inds]
                        t=t[inds_cfg]
                        t=np.mean(t,axis=2)
                        return t
                    
                    dat=np.transpose([get(mom) for mom in moms],[1,2,0])
                    dat=dat[:,:tf+1]
                    
                    weight=tf2Nsrcs[tf][ipath]/np.sum(tf2Nsrcs[tf])
                    tf2dat[tf]+=dat*weight
                # break
        # break
    
    
    path=f'{savepath}conn_2pt.h5'
    with h5py.File(path,'w') as f:
        f.create_dataset('notes',data=['cfg,time,mom'])
        f.create_dataset('cfgs',data=cfgs)
        f.create_dataset('moms',data=moms)
        
        for tf in tf2dat.keys():
            f.create_dataset(f'data/{tf}',data=np.real(tf2dat[tf]))

max_mom2=54
range_xyz=range(-int(np.sqrt(max_mom2))-1,int(np.sqrt(max_mom2))+2)
moms=[[0,0,0,x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2]       
moms=list(set([tuple(mom2standard(mom)) for mom in moms]))
moms=[list(mom[3:]) for mom in moms]; moms.sort()

run(moms)